In [ ]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

In [ ]:
import rawpy
import imageio.v3 as iio

path = 'data/raw-images/477A9296.CR3'
with rawpy.imread(path) as raw:
    rgb = raw.postprocess()
iio.imwrite('default.png', rgb)

In [37]:
from PIL import Image
import numpy as np
def raw_to_png(raw_file_path, width, height, output_path):
    """
    Converts a raw image file to PNG format.

    Args:
        raw_file_path (str): Path to the raw image file.
        width (int): Width of the image in pixels.
        height (int): Height of the image in pixels.
        output_path (str): Path to save the output PNG file.
    """
    try:
        with open(raw_file_path, 'rb') as f:
            raw_data = f.read()

        # Assuming 8-bit grayscale raw data, adjust accordingly if different
        image_array = np.frombuffer(raw_data, dtype=np.uint8).reshape((height, width))

        image = Image.fromarray(image_array, mode='L')  # 'L' for grayscale
        image.save(output_path)
        print(f"Image saved to {output_path}")

    except FileNotFoundError:
            print(f"Error: Raw file not found at {raw_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
raw_file = 'data/raw-images/477A9296.CR3'
img_width = 6720  # Replace with your image width
img_height = 4480 # Replace with your image height
output_png = 'data/png-conversions/image.png'

raw_to_png(raw_file, img_width, img_height, output_png)

An error occurred: cannot reshape array of size 48168115 into shape (4480,6720)


In [38]:
def concatenate_images(image_paths, orientation='horizontal'):
    """
    Concatenates images horizontally or vertically.

    Args:
        image_paths: A list of paths to the images to concatenate.
        orientation: 'horizontal' or 'vertical'.

    Returns:
        A new Image object containing the concatenated images.
    """
    images = [Image.open(path) for path in image_paths]
    widths, heights = zip(*(i.size for i in images))

    if orientation == 'horizontal':
        total_width = sum(widths)
        max_height = max(heights)
        new_image = Image.new('RGB', (total_width, max_height))
        x_offset = 0
        for image in images:
            new_image.paste(image, (x_offset, 0))
            x_offset += image.width
    elif orientation == 'vertical':
        total_height = sum(heights)
        max_width = max(widths)
        new_image = Image.new('RGB', (max_width, total_height))
        y_offset = 0
        for image in images:
            new_image.paste(image, (0, y_offset))
            y_offset += image.height
    else:
         raise ValueError("Orientation must be 'horizontal' or 'vertical'")
    return new_image

def create_image_grid(image_paths, images_per_row):
    """
    Creates a grid of images.

    Args:
        image_paths: A list of paths to the images to include in the grid.
        images_per_row: The number of images in each row.

    Returns:
         A new Image object containing the grid of images.
    """
    images = [Image.open(path) for path in image_paths]
    widths, heights = zip(*(i.size for i in images))
    max_width = max(widths)
    max_height = max(heights)
    
    rows = (len(images) + images_per_row - 1) // images_per_row
    
    new_image = Image.new('RGB', (images_per_row * max_width, rows * max_height))

    for index, image in enumerate(images):
        row_index = index // images_per_row
        col_index = index % images_per_row
        x_offset = col_index * max_width
        y_offset = row_index * max_height
        new_image.paste(image.resize((max_width, max_height)), (x_offset, y_offset))
    return new_image

In [46]:
from PIL import Image
from diffusers import LDMSuperResolutionPipeline
import torch

device = "mps" if torch.mps.is_available() else "cpu"
print(f"Using device: {device}")
model_id = "CompVis/ldm-super-resolution-4x-openimages"

# pipe = DiffusionPipeline.from_pretrained("CompVis/ldm-super-resolution-4x-openimages", force_download=True)

# load model and scheduler
pipeline = LDMSuperResolutionPipeline.from_pretrained(model_id, variant="fp16")
pipeline = pipeline.to(device)

# let's download an  image
image_height = 4480
image_width = 6720
subsection_dim = 160
n_rows = image_height // subsection_dim
n_cols = image_width // subsection_dim

print(f"Image size: {image_width} x {image_height}")
print(f"Subsection size: {subsection_dim} x {subsection_dim}")
print(f"Number of rows: {n_rows}")
print(f"Number of cols: {n_cols}")
image_path = "data/png-conversions/477A9285.png"
image = Image.open(image_path).convert("RGB")

upscaled_image_subsections = [[None for _ in range(n_cols)] for _ in range(n_rows)]

for i in range(n_rows):
    for j in range(n_cols):
        print(f"Processing subsection {i}, {j}")

        crop_box = (j * subsection_dim, i * subsection_dim, (j + 1) * subsection_dim, (i + 1) * subsection_dim)
        image_subsection = image.crop(crop_box)
        upscaled_image_subsection = pipeline(image_subsection, num_inference_steps=100, eta=1).images[0]
        upscaled_image_subsection.save(f"out/ldm_generated_image_{i:06}_{j:06}.png")
        # upscaled_image_subsections[i][j] = upscaled_image_subsection



# print(f"Image size: {image_width} x {image_height}")
# image_path = "data/png-conversions/477A9285.png"
# image = Image.open(image_path) \
#     .convert("RGB") \
#     .resize((image_width, image_height)) \

# # run pipeline in inference (sample random noise and denoise)
# upscaled_image = pipeline(image, num_inference_steps=100, eta=1).images[0]
# # save image
# upscaled_image.save("ldm_generated_image.png")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Using device: mps


Loading pipeline components...: 100%|██████████| 3/3 [00:00<00:00, 30.75it/s]


Image size: 6720 x 4480
Subsection size: 160 x 160
Number of rows: 28
Number of cols: 42
Processing subsection 0, 0


100%|██████████| 100/100 [00:10<00:00,  9.37it/s]


...

Processing subsection 0, 16


100%|██████████| 100/100 [00:10<00:00,  9.83it/s]


Processing subsection 0, 17


  6%|▌         | 6/100 [00:00<00:13,  7.10it/s]


KeyboardInterrupt: 

In [47]:
import os

image_chunk_files = os.listdir("out")
image_chunk_files = [os.path.join("out", file) for file in image_chunk_files]
image_chunk_files = [file for file in image_chunk_files if file.endswith('.png')]
image_chunk_files.sort()

# Create image grid
image_grid = create_image_grid(image_chunk_files, images_per_row=n_cols)
image_grid = image_grid.resize((image_width, image_height))
image_grid.save('image_grid.jpg')
